## The first cell will only execute if you're using Google Colab AND have not cloned the repository yet

In [1]:
import os
import sys
import subprocess

# Check if running in Google Colab
colab_setup = "google.colab" in sys.modules

repo_url = "https://github.com/sprouse9/URI_CapstoneProject.git"
repo_name = "URI_CapstoneProject"

# Clone only if running in Google Colab. Prevent cloning repo if already cloned.
if colab_setup and not os.path.exists('../' + repo_name):
    # Clone the repository if not already cloned

    if not os.path.exists(repo_name):
        print(f"Cloning repository: {repo_url}...")
        !git clone {repo_url}

    # Change directory to the repository
    %cd {repo_name}

#### This next cell takes care of the dataset download from my Google Drive as a zip file.  
#### The zip file will be auto extracted to your local machine or instance of Colab.
#### The download will not occur if the zip file or the extracted folder already exists.
#### The data folder 'archive' will not be unzipped again if already exists

In [2]:
import zipfile
import importlib.util

zip_filename = "CarDetectionDataSet.zip"
extract_folder = "archive"  # Define the folder where files are extracted

print(f"Working dir: {os.getcwd()}")  # This shows your current working directory

# Download only if the zip file and extracted folder don't exist
if not os.path.exists(zip_filename) and not os.path.exists(extract_folder):
    # Check if gdown is installed before attempting to install
    if importlib.util.find_spec("gdown") is None:
        print("gdown not found. Installing...")
        !pip install gdown

    print(f"{zip_filename} not found. Downloading...")
    !gdown 1JFAfrbUfXtiF-xwko2ACB-snDwIsj31h -O {zip_filename}
else:
    print(f"Skipping download. {zip_filename} or {extract_folder} already exists.")

# Extract only if the extracted folder does not exist
if not os.path.exists(extract_folder):
    print(f"Extracting {zip_filename}...")
    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        zip_ref.extractall()
    print(f"Extraction complete: {extract_folder}")
else:
    print(f"{extract_folder} already exists. Skipping extraction.")


Working dir: C:\Users\RyzenGaming\source\repos\URI_CapstoneProject
Skipping download. CarDetectionDataSet.zip or archive already exists.
archive already exists. Skipping extraction.


## The data folder has been setup.

In [3]:
if importlib.util.find_spec("ultralytics") is None:
    print("gdown not found. Installing...")
    !pip install -q ultralytics

if importlib.util.find_spec("torch") is None:
    print("torch not found. Installing...")
    !pip install -q torch

import torch
print("PyTorch Version:", torch.__version__)

PyTorch Version: 2.6.0+cu118


In [4]:
# take care of library related path issues regardless of the
# Operating system used

import yaml

# Load the existing dataset.yaml configuration
with open('dataset.yaml', 'r') as f:
    dataset = yaml.safe_load(f)

# Get the current working directory
cwd = os.getcwd()

# Dynamically adjust the 'path'
# This sets an absolute path for ultralytics so that it resolves correctly
# Shouldn't be necessary but here we are
dataset['path'] = os.path.join(cwd, 'archive')

print("Resolved dataset path:", dataset['path'])

with open('dataset_updated.yaml', 'w') as f:
    yaml.dump(dataset, f)

Resolved dataset path: C:\Users\RyzenGaming\source\repos\URI_CapstoneProject\archive


In [5]:
# Get GPU name and VRAM

# Set the device: use "cuda" if available, otherwise "cpu"
device = "cuda" if torch.cuda.is_available() else "cpu"

# Optionally, get VRAM information if using a GPU
if device == "cuda":
    vram = torch.cuda.get_device_properties(0).total_memory / 1e9  # Convert bytes to GB
    gpu_name = torch.cuda.get_device_name(0)
    print(f"Using GPU ({gpu_name}) with {vram:.2f} GB VRAM")
else:
    vram = None
    print("Using CPU")

#print("GPU Name:", gpu_name)
print("GPU VRAM:", torch.cuda.get_device_properties(0).total_memory / (1024**3), "GiB")
print(f"Available VRAM: {torch.cuda.memory_reserved(0) / 1e9:.2f} GB")

Using GPU (NVIDIA GeForce RTX 3060 Ti) with 8.59 GB VRAM
GPU VRAM: 7.9993896484375 GiB
Available VRAM: 0.00 GB


In [ ]:
import pandas as pd
from ultralytics import YOLO

if device == "cuda":
    learning_rates = [0.0005, 0.002, 0.01]
    epochs_list = [10, 20, 50]
    image_sz = 640
    # Using a tiered approach based on available VRAM:
    if vram >= 15:
        print("VRAM is very large")
        batch_sz = 64
    elif vram > 8:
        print("VRAM is large")
        batch_sz = 32  # a moderate increase
    else:
        batch_sz = 16
else:
    learning_rates = [0.0005, 0.002]
    epochs_list = [10]
    image_sz = 320
    batch_sz = 4

# List to store results from each experiment
results_summary = []

for lr in learning_rates:
    for epochs in epochs_list:
        print(f"Training with lr: {lr}, epochs: {epochs}, imgsz:{image_sz}")

        # Conditionally clear CUDA memory if available
        if device == "cuda":
          torch.cuda.empty_cache()
          torch.cuda.ipc_collect()
        else:
          print("CUDA not available; skipping CUDA memory cleanup.")

        # Initialize the model
        model = YOLO("yolov8n.pt")

        # Train the model with the current hyperparameters
        results = model.train(
            data="dataset_updated.yaml",
            epochs=epochs,
            batch=batch_sz,
            imgsz=image_sz,
            lr0=lr,
            #cache=True,
            cache='disk',
            optimizer="AdamW",
            project=f"runs/train/lr{lr}_ep{epochs}_{image_sz}"
        )

        # Get the results dictionary
        rdict = results.results_dict  # Contains keys like 'metrics/precision(B)' etc.

        # Append the metrics along with the hyperparameters to our list
        results_summary.append({
            "lr": lr,
            "epochs": epochs,
            "precision": rdict.get("metrics/precision(B)", None),
            "recall": rdict.get("metrics/recall(B)", None),
            "mAP50": rdict.get("metrics/mAP50(B)", None),
            "mAP50-95": rdict.get("metrics/mAP50-95(B)", None),
            "fitness": rdict.get("fitness", None)
        })


        # Conditionally clear CUDA memory if available
        # if torch.cuda.is_available():
        #    torch.cuda.empty_cache()
        #    torch.cuda.ipc_collect()
        # else:
        #     print("CUDA not available; skipping CUDA memory cleanup.")


# Convert the results list into a DataFrame for easy viewing
results_df = pd.DataFrame(results_summary)
print("\nBaseline Metrics for Each Test:")
print(results_df)

### Baseline Metrics for Each Test:

|   lr   | epochs | precision |  recall  |   mAP50   | mAP50-95 | fitness |
|--------|---------|-----------|----------|-----------|-----------|----------|
| 0.0005 |    10   | 0.928012  | 0.562112 | 0.821413  | 0.609509  | 0.630699 |
| 0.0005 |    20   | 0.849446  | 0.784161 | 0.892621  | 0.694569  | 0.714374 |
| 0.0005 |    50   | 0.829223  | 0.821429 | 0.907031  | 0.726589  | 0.744633 |
| 0.0020 |    10   | 0.945445  | 0.618930 | 0.849143  | 0.664448  | 0.682917 |
| 0.0020 |    20   | 0.832686  | 0.803705 | 0.895060  | 0.718127  | 0.735820 |
| 0.0020 |    50   | 0.854076  | 0.808862 | 0.909397  | 0.750816  | 0.766674 |
| 0.0100 |    10   | 0.870470  | 0.739130 | 0.862304  | 0.596275  | 0.622878 |
| 0.0100 |    20   | 0.917089  | 0.754658 | 0.879957  | 0.705703  | 0.723128 |
| 0.0100 |    50   | 0.949638  | 0.768634 | 0.927499  | 0.777226  | 0.792253 |


In [ ]:
import torch
from ultralytics import YOLO

# Check if GPU is available
#device = "cuda" if torch.cuda.is_available() else "cpu"
#print("Device set to:", device)

# Initialize the YOLOv8n model
model = YOLO("yolov8n.pt")

# Define a search space for hyperparameter tuning
# You can add or remove parameters as needed.
search_space = {
    "lr0": (1e-5, 1e-1),            # Initial learning rate
    "degrees": (0.0, 45.0),         # Rotation augmentation range
    "momentum": (0.6, 0.98),        # Momentum for SGD
    "weight_decay": (0.0, 0.001),   # L2 regularization
    "scale": (0.0, 0.9)             # Scaling augmentation range
}

# Perform hyperparameter tuning using the Tuner class
# Note: 'data' should point to your dataset configuration file
results = model.tune(
    data="dataset_updated.yaml",
    epochs=40,             # Train for this many epochs per tuning iteration
    iterations=50,         # Number of tuning iterations (adjust as necessary)
    optimizer="AdamW",     # Use the AdamW optimizer
    space=search_space,    # The defined search space
    plots=True,            # Skip plotting for faster tuning
    save=True,             # Skip checkpointing to reduce overhead
    val=True               # Skip validation until the final epoch for faster tuning
)

# Print out the best hyperparameters found
print("Tuning complete. Best hyperparameters:")
print(results.best_hyperparameters)


In [ ]:
results.best_hyperparameters

In [ ]:
# using the best parameter found, let's train the model with the image size to the actual 1920x1080

# List to store results from each experiment
results_summary = []

# Conditionally clear CUDA memory if available
if device == "cuda":
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()
else:
    print("CUDA not available; skipping CUDA memory cleanup.")

# Initialize the model
model = YOLO("yolov8n.pt")

lr = 0.01
epochs=50

# Train the model with the current hyperparameters
results = model.train(
        data="dataset_updated.yaml",
        epochs=50,
        batch=8,
        imgsz=1920,
        lr0=lr,
        #cache=True,
        cache='disk',
        optimizer="AdamW",
        project=f"runs/train/lr{lr}_ep{epochs}_imgsz1920"
        )

# Get the results dictionary
rdict = results.results_dict  # Contains keys like 'metrics/precision(B)' etc.

# Append the metrics along with the hyperparameters to our list
results_summary.append({
            "lr": 0.01,
            "epochs": epochs,
            "precision": rdict.get("metrics/precision(B)", None),
            "recall": rdict.get("metrics/recall(B)", None),
            "mAP50": rdict.get("metrics/mAP50(B)", None),
            "mAP50-95": rdict.get("metrics/mAP50-95(B)", None),
            "fitness": rdict.get("fitness", None)
        })


# Convert the results list into a DataFrame for easy viewing
results_df = pd.DataFrame(results_summary)
print("\nBaseline Metrics for Each Test:")
print(results_df)


In [4]:
# Check if the results object exists and display the best hyperparameters
if 'results' in locals():
    print("Tuning complete. Best hyperparameters:")
    print(results.best_hyperparameters)
else:
    print("The 'results' object is not available.")


The 'results' object is not available.


In [7]:
import pandas as pd

# Load and display the tuning results from the CSV file
tune_results_path = "runs/detect/tune/tune_results.csv"
tune_results_df = pd.read_csv(tune_results_path)
print("\nTuning Results:")
print(tune_results_df.head())  # Display the first few rows



Tuning Results:
   fitness      lr0  degrees  momentum  weight_decay    scale
0  0.00000  0.01000      0.0   0.93700       0.00050  0.50000
1  0.00000  0.00887      0.0   0.98000       0.00051  0.52147
2  0.76813  0.01027      0.0   0.76892       0.00045  0.52030
3  0.77379  0.00779      0.0   0.76892       0.00053  0.49947
4  0.76679  0.01106      0.0   0.76892       0.00040  0.42310


In [8]:
import yaml

# Load and display the best hyperparameters from the YAML file
best_hyperparams_path = "runs/detect/tune/best_hyperparameters.yaml"
with open(best_hyperparams_path, 'r') as file:
    best_hyperparameters = yaml.safe_load(file)

print("\nBest Hyperparameters Found:")
print(best_hyperparameters)



Best Hyperparameters Found:
{'lr0': 0.01098, 'degrees': 0.0, 'momentum': 0.80203, 'weight_decay': 0.00039, 'scale': 0.33385}


In [ ]:
from ultralytics import YOLO

# Initialize the YOLO model
model = YOLO("yolov8n.pt")  # Replace with the appropriate YOLO model if needed

# Train using the best hyperparameters
results = model.train(
    data="dataset_updated.yaml",          
    epochs=50,                            # You can adjust epochs as needed
    batch=best_hyperparameters.get("batch", 8),  # 8 is the default if parameter not found
    imgsz=best_hyperparameters.get("imgsz", 640),
    lr0=best_hyperparameters.get("lr0", 0.002),
    optimizer=best_hyperparameters.get("optimizer", "AdamW"),
    project="runs/train_best_hyperparams",
    cache='disk'
)

# Display the results
print("\nTraining Complete. Results:")
print(results)


In [16]:
results.results_dict

{'metrics/precision(B)': 0.9453313375058685,
 'metrics/recall(B)': 0.7670807453416149,
 'metrics/mAP50(B)': 0.9171910895522071,
 'metrics/mAP50-95(B)': 0.7776969412755597,
 'fitness': 0.7916463561032244}

Our fitness score with the tuned model is 0.7916 vs 0.79225 for the best model in our gridsearch.

In [6]:
# using the best parameters found, let's train the model with the image size to the actual 1920x1080

from ultralytics import YOLO

# List to store results from each experiment
results_summary = []

# Conditionally clear CUDA memory if available
if device == "cuda":
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()
else:
    print("CUDA not available; skipping CUDA memory cleanup.")

# Initialize the model
model = YOLO("yolov8n.pt")

lr=0.01
epochs = 50
image_sz = 1920

# Train the model with the current hyperparameters
results = model.train(
        data="dataset_updated.yaml",
        epochs=epochs,
        batch=4,
        imgsz=image_sz,
        lr0=lr,
        #cache=True,
        cache='disk',
        optimizer="AdamW",
        project=f"runs/train/lr{lr}_ep{epochs}_imgsz{image_sz}"
        )

# Get the results dictionary
rdict = results.results_dict  # Contains keys like 'metrics/precision(B)' etc.

# Append the metrics along with the hyperparameters to our list
results_summary.append({
        "lr": lr,
        "epochs": epochs,
        "precision": rdict.get("metrics/precision(B)", None),
        "recall": rdict.get("metrics/recall(B)", None),
        "mAP50": rdict.get("metrics/mAP50(B)", None),
        "mAP50-95": rdict.get("metrics/mAP50-95(B)", None),
        "fitness": rdict.get("fitness", None)
        })


# Convert the results list into a DataFrame for easy viewing
results_df = pd.DataFrame(results_summary)
print("\nBaseline Metrics for Each Test:")
print(results_df)


New https://pypi.org/project/ultralytics/8.3.78 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.75  Python-3.9.21 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8191MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=dataset_updated.yaml, epochs=50, time=None, patience=100, batch=4, imgsz=1920, save=True, save_period=-1, cache=disk, device=None, workers=8, project=runs/train/lr0.01_ep50_imgsz1920, name=train5, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False,

train: Scanning C:\Users\RyzenGaming\source\repos\URI_CapstoneProject\archive\train\labels.cache... 400 images, 0 backgrounds, 0 corrupt: 100%|██████████| 400/
train: Caching images (2.3GB Disk): 100%|██████████| 400/400 [00:00<00:00, 15986.52it/s]
val: Scanning C:\Users\RyzenGaming\source\repos\URI_CapstoneProject\archive\val\labels.cache... 99 images, 0 backgrounds, 0 corrupt: 100%|██████████| 99/99 [00
val: Caching images (0.6GB Disk): 100%|██████████| 99/99 [00:00<00:00, 14120.31it/s]


Plotting labels to runs\train\lr0.01_ep50_imgsz1920\train5\labels.jpg... 
optimizer: AdamW(lr=0.01, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 1920 train, 1920 val
Using 8 dataloader workers
Logging results to runs\train\lr0.01_ep50_imgsz1920\train5
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50      5.34G     0.7661     0.9052     0.9199        100       1920: 100%|██████████| 100/100 [00:25<00:00,  4.00it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:01<00:00,  6.64it/s]

                   all         99        644        0.4      0.491      0.498      0.387


...

                   all         99        644      0.935      0.896      0.964      0.874



50 epochs completed in 0.319 hours.
Optimizer stripped from runs\train\lr0.01_ep50_imgsz1920\train5\weights\last.pt, 6.5MB
Optimizer stripped from runs\train\lr0.01_ep50_imgsz1920\train5\weights\best.pt, 6.5MB

Validating runs\train\lr0.01_ep50_imgsz1920\train5\weights\best.pt...
Ultralytics 8.3.75  Python-3.9.21 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8191MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:01<00:00,  7.24it/s]


                   all         99        644      0.957      0.888      0.973      0.875
Speed: 0.8ms preprocess, 5.7ms inference, 0.0ms loss, 2.4ms postprocess per image
Results saved to runs\train\lr0.01_ep50_imgsz1920\train5


NameError: name 'pd' is not defined

In [7]:
rdict

{'metrics/precision(B)': 0.9572013569173281,
 'metrics/recall(B)': 0.8881987577639752,
 'metrics/mAP50(B)': 0.9728524568695167,
 'metrics/mAP50-95(B)': 0.8749807813921849,
 'fitness': 0.8847679489399181}